# 07b: Extended sensitivity analysis

In this script, we perform sensitivity analysis by sampling technosphere, biosphere, and weighting (monetization) inputs.

In [1]:
from lca_sensitivity_analysis import *

import xarray as xr
from scipy import sparse
from time import time

import seaborn as sns
import matplotlib.pyplot as plt

import multiprocessing as mp

In [2]:
%run common_definitions.py

In [3]:
bw.projects.set_current(BW_PROJECTNAME)
output_fp = "../output/" + BW_PROJECTNAME

### Select scenario and year and further parameter choices

In [4]:
scen = "SSP2-NPi"
year = 2030
substring = "{}_{}".format(scen, str(year))
db = bw.Database([name for name in bw.databases if substring in name][0])

### Select technologies

In [5]:
df = pd.read_csv("../mappings/technology_selection_energy_provision_v4.csv")
long2short = dict(zip(list(df["ecoinvent name"]), list(df["short name"])))

In [6]:
subset_names = list(df["ecoinvent name"].unique())
db_subset = [act for act in db if act["name"] in subset_names]

In [7]:
def contains_global(actlist):
    locations = [act["location"] for act in actlist]

    return any(x in locations for x in ["RoW", "GLO", "World"])

def select_region(actlist):
    locations = [act["location"] for act in actlist]

    if contains_global(actlist):
        if "GLO" in locations:
            return actlist[locations.index("GLO")]
        elif "World" in locations:
            return actlist[locations.index("World")]
        else:
            return actlist[locations.index("RoW")]
    else:
        # choose alphabetically first location (need a deterministic selection)
        first_location = sorted(locations)[0]
        return actlist[locations.index(first_location)]
    


In [8]:
activities = {}

for sector in df["sector"].unique():
    sel = df[df["sector"] == sector]
    all_matches = []
    for name, product in zip(list(sel["ecoinvent name"]), list(sel["reference product"])):
        matches = [act for act in db_subset if act["name"] == name and act["reference product"] == product]
        if len(matches) == 0:
            print("No matches for {}, {}".format(name, product))
        sel = select_region(matches)
        # if not contains_global(matches):
        #     print("No global dataset for {}, {}. Location {} selected.".format(name, product, sel["location"]))
        # print("\t{} activities found for {}, {} in database {}".format(len(matches), name, product, substring))
        all_matches.append(sel)
    print("A total of {} activities found in sector {} in database {}".format(len(all_matches), sector, substring))
    activities[sector] = all_matches

A total of 45 activities found in sector electricity production in database SSP2-NPi_2030
A total of 17 activities found in sector district or industrial heat in database SSP2-NPi_2030
A total of 48 activities found in sector liquids in database SSP2-NPi_2030
A total of 18 activities found in sector hydrogen and gases in database SSP2-NPi_2030
A total of 9 activities found in sector solids in database SSP2-NPi_2030
A total of 12 activities found in sector residential heating in database SSP2-NPi_2030


### Get methods and weights (monetization factors)

In [9]:
mfs_mc_sample = xr.open_dataarray("../data/mfs_monte_carlo_sample_euro{}_larger.nc".format(str(EURO_REF_YEAR)))

In [10]:
weights_total = mfs_mc_sample.sum(dim="impact category")

In [11]:
methods = [m for m in list(bw.methods) if ", ".join(list(m)) in weights_total.coords["LCIA method"].values]

ALL_WEIGHTS = weights_total.to_pandas().T.loc[[", ".join(list(m)) for m in methods]].to_numpy()
mean_weights = np.mean(ALL_WEIGHTS, axis=1)

## One-factor at a time analysis

In [ ]:
def gsa_ofat(act, methods, mean_weights, Nsamples, print_to_console=True):
    # set up LCA
    fu = {act: 1}
    lca = bw.LCA(fu)
    lca.lci()

    # biosphere variation
    t0 = time()
    bio_rng = MCRandomNumberGenerator(lca.bio_params)
    bio_results = []
    for i in range(Nsamples):
        lca.rebuild_biosphere_matrix(bio_rng.next())

        count = len(lca.activity_dict)
        lca.inventory = lca.biosphere_matrix * \
            sparse.spdiags([lca.supply_array], [0], count, count)
        
        temp = []
        for m in methods:
            lca.switch_method(m)
            lca.lcia_calculation()
            temp.append(lca.score)

        bio_results.append(np.sum(np.array(temp) * mean_weights))
        print("{}: Biosphere: Run {} of {} done.".format(str(act), i+1, Nsamples))
    t1 = time()

    if print_to_console:
        print("{}: Biosphere variation runs done in {} seconds".format(str(act), round(t1-t0, 2)))

    # technosphere variation
    t0 = time()
    tech_rng = MCRandomNumberGenerator(lca.tech_params)
    tech_results = []
    for i in range(Nsamples):
        t2 = time()
        lca.rebuild_technosphere_matrix(tech_rng.next())

        t3 = time()

        lca.redo_lci(fu)

        t4 = time()

        temp = []
        for m in methods:
            lca.switch_method(m)
            lca.lcia_calculation()
            temp.append(lca.score)

        tech_results.append(np.sum(np.array(temp) * mean_weights))
        print("{}: Technosphere: Run {} of {} done (rebuilding: {} seconds, lci: {} seconds)".format(str(act), i+1, Nsamples, t3-t2, t4-t3))
    t1 = time()

    if print_to_console:
        print("{}: Technosphere variation runs done in {} seconds".format(str(act), t1-t0))

    return {
        "technosphere": tech_results,
        "biosphere": bio_results
    }

In [ ]:
def gsa_ofat_multi(actlist, mean_cost_method, Nsamples):
    # set up LCA
    fu = {actlist[0]: 1}
    lca = bw.LCA(fu, mean_cost_method)
    lca.lci()
    lca.lcia()

    ## Biosphere variation ##
    # calculate supply arrays for all activities
    lca.decompose_technosphere()
    supply_arrays = []
    count = len(lca.activity_dict)
    for act in actlist:
        lca.redo_lci({act: 1})
        supply_arrays.append(lca.supply_array)

    bio_rng = MCRandomNumberGenerator(lca.bio_params)
    bio_results = np.zeros((len(actlist), Nsamples))
    for j in range(Nsamples):
        lca.rebuild_biosphere_matrix(bio_rng.next())
        for i, supply in enumerate(supply_arrays):
            lca.inventory = lca.biosphere_matrix * \
                    sparse.spdiags([supply], [0], count, count)
            lca.lcia_calculation()
            bio_results[i, j] = lca.score
            print("Biosphere variation: Activity {} of {}, run {} of {} done.".format(i+1, len(actlist), j+1, Nsamples))

    # technosphere variation
    # set up LCA again
    fu = {actlist[0]: 1}
    lca = bw.LCA(fu, mean_cost_method)
    lca.lci()
    lca.lcia()

    tech_rng = MCRandomNumberGenerator(lca.tech_params)
    tech_results = np.zeros((len(actlist), Nsamples))
    for j in range(Nsamples):
        lca.rebuild_technosphere_matrix(tech_rng.next())
        lca.decompose_technosphere()

        for i, act in enumerate(actlist):
            lca.redo_lci({act: 1})
            lca.lcia_calculation()
            tech_results[i, j] = lca.score
            print("Technosphere variation: Activity {} of {}, Run {} of {} done.".format(i+1, len(actlist), j+1, Nsamples))

    return {
        "technosphere": tech_results,
        "biosphere": bio_results
    }

In [12]:
cost_method = ("Monte Carlo monetization (larger sample)", "mean total costs")

In [13]:
all_activities = []
all_sectors = []
for sec, a in activities.items():
    all_activities += a
    all_sectors += len(a) * [sec,]

In [14]:
short2location = {}
for act in all_activities:
    tech = long2short[act["name"]]
    loc = act["location"]
    short2location[tech] = loc

In [ ]:
scores_multi_all = gsa_ofat_multi(all_activities, cost_method, 2000)

In [ ]:
dflist = []
for stage, results in scores_multi_all.items():
    for i, act in enumerate(all_activities):
        y = results[i,:]
        df = pd.DataFrame({
            "output value": y,
        })
        df["stage"] = stage
        df["sample index"] = np.arange(len(y))
        df["short name"] = long2short[act["name"]]
        df["sector"] = all_sectors[i]
        df["scenario"] = scen
        df["year"] = year
        df["location"] = act["location"]
        dflist.append(df)

In [ ]:
output = pd.concat(dflist, axis=0, ignore_index=True)
output.to_csv("gsa_ofat_all_sample1.csv")

### Calculate costs

In [15]:
new_cs = {"inv": [{act: 1} for act in all_activities], "ia": methods}
bw.calculation_setups["GSA"] = new_cs

mc = bw.MultiLCA("GSA")
    

In [16]:
costs = np.matmul(mc.results, weights_total.to_pandas()[[", ".join(list(m)) for m in methods]].to_numpy().T)

In [17]:
idx = []
for act, sector in zip(all_activities, all_sectors):
    tech = long2short[act["name"]]
    idx.append(str((tech, sector)))

In [21]:
xr.DataArray(
    costs,
    coords={
        "activity key": idx,
        "sample index": list(range(costs.shape[1]))
    }
).to_netcdf(output_fp+"/gsa_ofat_all_costs.nc")

### Calculate variances and sensitivity scores

In [22]:
output = pd.read_csv(output_fp+"/gsa_ofat_all_outputsamples.csv")
output

,output value,stage,sample index,short name,sector,scenario,year,location
0,0.102113,technosphere,0,BIGCC,electricity production,SSP2-NPi,2030,World
1,0.105272,technosphere,1,BIGCC,electricity production,SSP2-NPi,2030,World
2,0.102952,technosphere,2,BIGCC,electricity production,SSP2-NPi,2030,World
3,0.101132,technosphere,3,BIGCC,electricity production,SSP2-NPi,2030,World
4,0.100353,technosphere,4,BIGCC,electricity production,SSP2-NPi,2030,World
...,...,...,...,...,...,...,...,...
595995,0.021843,biosphere,1995,diffusion adsorption heat pump,residential heating,SSP2-NPi,2030,World
595996,0.025134,biosphere,1996,diffusion adsorption heat pump,residential heating,SSP2-NPi,2030,World
595997,0.022911,biosphere,1997,diffusion adsorption heat pump,residential heating,SSP2-NPi,2030,World
595998,0.018084,biosphere,1998,diffusion adsorption heat pump,residential heating,SSP2-NPi,2030,World


In [23]:
def filter_by_quantiles(df, idx, qidx, col, low, high):
    dfnew = df.set_index(idx)
    low = df.groupby(qidx)[col].quantile(low).rename("low")
    high = df.groupby(qidx)[col].quantile(high).rename("high")

    dfnew = dfnew.join(low)
    dfnew = dfnew.join(high).reset_index()

    sel = dfnew[dfnew[col] <= dfnew["high"]]
    sel = sel[sel[col] >= sel["low"]]

    return sel

def calculate_variances(df, chunks=1):
    dfnew = df.copy()
    Nsamples = dfnew["sample index"].max() + 1
    dfnew["chunk index"] = dfnew["sample index"] // (Nsamples // chunks)
    
    return dfnew.groupby(["short name", "sector", "stage", "chunk index"]).agg({"output value": np.var})

def calculate_stds(df, chunks=1):
    dfnew = df.copy()
    Nsamples = dfnew["sample index"].max() + 1
    dfnew["chunk index"] = dfnew["sample index"] // (Nsamples // chunks)
    
    return dfnew.groupby(["short name", "sector", "stage", "chunk index"]).agg({"output value": np.std})

def normalize_column(df, col, values, new_value_name):
    idx = list(set(df.columns) - {col}- {values})
    a = df.pivot(index=idx, columns=col, values=values)

    return a.divide(a.sum(axis=1), axis=0).melt(
        ignore_index=False, value_name=new_value_name).reset_index()

def cv(x):
    return np.std(x) / np.mean(x)

def calculate_cvs(df, chunks=1):
    dfnew = df.copy()
    Nsamples = dfnew["sample index"].max() + 1
    dfnew["chunk index"] = dfnew["sample index"] // (Nsamples // chunks)
    
    return dfnew.groupby(["short name", "sector", "stage", "chunk index"]).agg({"output value": cv})



In [24]:
filter_levels = [0, 0.01, 0.02, 0.05, 0.1]

dflist = []
for fl in filter_levels:
    filtered_data = filter_by_quantiles(
                                output,
                                ["short name", "sector", "stage", "sample index"],
                                ["short name", "sector", "stage"],
                                "output value",
                                fl,
                                1-fl)
    
    df_var1 = calculate_variances(filtered_data).reset_index().drop(columns="chunk index").rename(columns={"output value": "variance"})
    df_var2 = pd.DataFrame({
        "variance": np.var(costs, axis=1),
        "short name": [long2short[act["name"]] for act in all_activities],
        "sector": all_sectors
    })
    df_var2["stage"] = "monetization"

    df_var = pd.concat((df_var1, df_var2), ignore_index=True)
    df_var["scenario"] = scen
    df_var["year"] = year
    df_var["location"] = df_var["short name"].apply(lambda x: short2location[x])

    df_sens = normalize_column(
        df_var, "stage", "variance", "sensitivity"
        ).set_index(
            ["short name", "sector", "stage"]
            ).sort_index().reset_index()
    
    df_sens["filter level"] = fl
    dflist.append(df_sens)

df_sens_all = pd.concat(dflist, ignore_index=False)
df_sens_all.to_csv(output_fp+"/gsa_ofat_all_sensdata_w_filters.csv", index=False)

C:\Users\davidba\AppData\Local\Temp\ipykernel_13456\2324570165.py:19: FutureWarning: The provided callable <function var at 0x000001AB14D3F880> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  return dfnew.groupby(["short name", "sector", "stage", "chunk index"]).agg({"output value": np.var})
C:\Users\davidba\AppData\Local\Temp\ipykernel_13456\2324570165.py:19: FutureWarning: The provided callable <function var at 0x000001AB14D3F880> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  return dfnew.groupby(["short name", "sector", "stage", "chunk index"]).agg({"output value": np.var})
C:\Users\davidba\AppData\Local\Temp\ipykernel_13456\2324570165.py:19: FutureWarning: The provided callable <function var at 0x000001AB14D3F880> is currently using Ser

In [25]:
filter_levels = [0, 0.01, 0.02, 0.05, 0.1]

dflist = []
for fl in filter_levels:
    filtered_data = filter_by_quantiles(
                                output,
                                ["short name", "sector", "stage", "sample index"],
                                ["short name", "sector", "stage"],
                                "output value",
                                fl,
                                1-fl)
    
    df_cv1 = calculate_cvs(filtered_data).reset_index().drop(columns="chunk index").rename(columns={"output value": "CV"})
    df_cv2 = pd.DataFrame({
        "CV": np.apply_along_axis(cv, 1, costs),
        "short name": [long2short[act["name"]] for act in all_activities],
        "sector": all_sectors
    })
    df_cv2["stage"] = "monetization"

    df_cv = pd.concat((df_cv1, df_cv2), ignore_index=True)
    df_cv["scenario"] = scen
    df_cv["year"] = year
    df_cv["location"] = df_var["short name"].apply(lambda x: short2location[x])
    
    df_cv["filter level"] = fl
    dflist.append(df_cv)

df_cv_all = pd.concat(dflist, ignore_index=False)
df_cv_all.to_csv(output_fp+"/gsa_ofat_all_cvdata_w_filters.csv", index=False)

In [ ]:
def calculate_rolling_variances(df):
    Nsamples = df["sample index"].max() + 1
    
    return df.groupby(["short name", "sector", "stage"]).rolling(Nsamples, min_periods=2).var()

Without quantile filtering

In [57]:
df_var1 = calculate_variances(output).reset_index().drop(columns="chunk index").rename(columns={"output value": "variance"})

C:\Users\davidba\AppData\Local\Temp\ipykernel_25628\3366105626.py:6: FutureWarning: The provided callable <function var at 0x000001627BA6F880> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  return dfnew.groupby(["short name", "sector", "stage", "chunk index"]).agg({"output value": np.var})


In [58]:
df_var2 = pd.DataFrame({
    "variance": np.var(costs, axis=1),
    "short name": [long2short[act["name"]] for act in all_activities],
    "sector": all_sectors
})
df_var2["stage"] = "monetization"

In [66]:
df_var = pd.concat((df_var1, df_var2), ignore_index=True)
df_var["scenario"] = scen
df_var["year"] = year
df_var["location"] = df_var["short name"].apply(lambda x: short2location[x])
df_var.to_csv(output_fp+"/gsa_ofat_all_vardata.csv", index=False)

In [ ]:
df_sens = normalize_column(
        df_var, "stage", "variance", "sensitivity"
        ).set_index(
            ["short name", "sector", "stage"]
            ).sort_index().reset_index()

In [75]:
df_sens.to_csv(output_fp+"/gsa_ofat_all_sensdata.csv", index=False)

### Also try standard deviation instead of variance

In [24]:
df_std1 = calculate_stds(output).reset_index().drop(columns="chunk index").rename(columns={"output value": "std"})

C:\Users\davidba\AppData\Local\Temp\ipykernel_18208\2285359168.py:26: FutureWarning: The provided callable <function std at 0x000002740C05B760> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  return dfnew.groupby(["short name", "sector", "stage", "chunk index"]).agg({"output value": np.std})


In [26]:
df_std2 = pd.DataFrame({
    "std": np.std(costs, axis=1),
    "short name": [long2short[act["name"]] for act in all_activities],
    "sector": all_sectors
})
df_std2["stage"] = "monetization"

In [27]:
df_std = pd.concat((df_std1, df_std2), ignore_index=True)
df_std["scenario"] = scen
df_std["year"] = year
df_std["location"] = df_std["short name"].apply(lambda x: short2location[x])
df_std.to_csv(output_fp+"/gsa_ofat_all_stddata.csv", index=False)

In [29]:
df_sens_std = normalize_column(
        df_std, "stage", "std", "sensitivity"
        ).set_index(
            ["short name", "sector", "stage"]
            ).sort_index().reset_index()

In [30]:
df_sens_std.to_csv(output_fp+"/gsa_ofat_all_sensdata_std.csv", index=False)